In [10]:
import numpy as np
from numpy import floor, sqrt
import scipy.sparse as sp
import scipy.sparse.linalg
import matplotlib.pyplot as plt
from math import pi as π
# import The_clash
# get_ipython().run_line_magic('matplotlib', 'widget')
from joblib import delayed, Parallel, parallel_backend
from Operators_warehouse import construct_basis_superfast, transform_new, diagonals_with_energies, erase
from architecture_copy import Exp_2qubits_2cavities_2purcells
from waveguide import Waveguide

## Survey on quantum trajectories 

Let's consider that we have a system, $H$, which suffers decoherence via the quantum jump operators $\{L_m\}$ with rates $\{\gamma_m\}$ in a purely Markovian manner, so that, the dynamics follows
\begin{align}
\dot{\rho}=-i\left[H,\rho\right]+\sum_m \gamma_m \left(L_m \rho L_m^\dagger-\frac{1}{2}\left\{L_m^\dagger L_m,\rho \right\}\right).
\end{align}

Although the previous equation can be numerically integrated, dealing with $\rho$ involves $N^2$ elements (being $N$ the Hilbert dimension). One can nevertheless resort to a stochastic dynamics resorting to quantum trajectories which involve only a pure quantum state (dimension $N$). The price to pay is the computation of ensemble average over many stochastic repetitions. 

A quantum trajectory can be computed following a non-Hermitian Hamiltonian, 
\begin{align}
    H_{\rm eff}=H-i\sum_m\frac{\gamma_m}{2}L_m^\dagger L_m
\end{align}
The evolution under $H_{\rm eff}$ does not preserve the state norm. During the evolution between $t\rightarrow t+dt$  the state undergoes a quantum jump with probability
\begin{align}
    P_m=\gamma_m dt \bra{\psi(t)}L_m^\dagger L_m\ket{\psi(t)}
\end{align}

If the quantum jump takes place, then
\begin{align}
    \ket{\psi(t+dt)}=\frac{L_m\ket{\psi(t)}}{|| L_m\ket{\psi(t)}||},
\end{align}
otherwise, the state evolves freely under $H_{\rm eff}$,
\begin{align}
    \ket{\psi(t+dt)}=\frac{e^{-i dt H_{\rm eff}}\ket{\psi(t)}}{||e^{-i dt H_{\rm eff}}\ket{\psi(t)}||}.
\end{align}
For each stochastic realization of a quantum trajectory, denoted by subscript $r$ we can compute the observable $A_r(t)=\bra{\psi_r(t)}A\ket{\psi_r(t)}$, so that
\begin{align}
    \overline{A}(t)=\frac{1}{N_r}\sum_{r=1}^{N_r}A_r(t)
\end{align}
In the limit, $N_r\rightarrow\infty$, $\overline{A}(t)={\rm Tr}[\rho(t) A]$ with $\rho(t)$ obtained from Eq.~\eqref{eq:ME}, and similarly for the state, $\overline{\rho}(t)=\sum_{r=1}^{N_r}\ket{\psi_r(t)}\bra{\psi_r(t)}/N_r$, which leads to $\overline{\rho}(t)=\rho(t)$ when $N_r\rightarrow \infty$. 

### Pseudocode

The method is quite simple, but there are some subtleties. In order to properly solve the dynamics, we require $P_m\leq 1$ and thus $\gamma_m dt\ll 1$. Follows the procedure to implement a quantum trajectory. 

- We start with the state $\ket{\psi(t)}$. 
- Throw a uniformly distributed random number $r\in[0,1]$ for each of the decay channels ($r_m$) and compute $P_m=\gamma_m dt \bra{\psi(0)}L_m^\dagger L_m \ket{\psi(0)}$. 
- If $r_m<P_m$ then there is a quantum jump with $L_m$, so that $\ket{\psi(t+dt)}=L_m\ket{\psi(t)}$
- Otherwise (no quantum jump), the state becomes $\ket{\psi(t+dt)}=e^{-i dt H_{\rm eff}}\ket{\psi(dt)}$
- Normalize the state. 
- Repeat now starting from $\ket{\psi(t+dt)}$ 
- Repeat the whole evolution $N_r$ times to get a reliable ensemble average


## Toy model example

Let's consider one simple example:
\begin{align}
    H=\sum_{i=1,2}\frac{\omega_i}{2}\sigma^z_i+\lambda \left( \sigma^+_ia+\sigma^-_ia^\dagger\right)+\omega_0 a^\dagger a
\end{align}
with decay channels $L_1=\sigma_1^-$ and $L_2=\sigma_2^-$. Hence, the effective non-Hermitian Hamiltonian is
\begin{align}
    H_{\rm eff}=H-i\sum_{i=1,2}\frac{\gamma_i}{2}\sigma^+_i\sigma^-_i
\end{align}

If we label the states $\ket{0}\ket{0}\ket{0}$, $\ket{1}\ket{0}\ket{0}$, $\ket{0}\ket{1}\ket{0}$, $\ket{0}\ket{0}\ket{1}$

In this limit, H preserves the number of excitations, but it does not preserve the norm (is that possible?)

### Implementation of the toy model

Instead of the cutre hard coded thing we have done, there is an alternative consisting on 

$$ \text{Basis} = \sigma_{z1} \otimes I_2 \otimes I_{\text{N*N}} +   I_1 \otimes \sigma_{z2}  \otimes I_{\text{N*N}}  +   I_1 \otimes  I_2  \otimes a^{\dagger}a_{\text{N*N}}  $$

First we have to define the small operators. And later we do the kroenecker products.

In [ ]:
def boson_creation(nmax, format='csr'):
    ad = np.sqrt(np.arange(1., nmax))
    if format == 'dense':
        return np.diag(ad, -1)
    else:
        return sp.diags(ad, -1, format=format)

def boson_anihilation(nmax, format='csr'):
    a = np.sqrt(np.arange(1., nmax))
    if format == 'dense':
        return np.diag(a, +1)
    else:
        return sp.diags(a, +1, format=format)

def boson_number(nmax, format='csr'):
    n = np.arange(0., nmax)
    if format == 'dense':
        return np.diag(0, +1)
    else:
        return sp.diags(n, 0, format=format)

In [ ]:
γ1 = 0.2
γ2 = 0.1
λ = 0.5
ω_0 = 1
ω_1 = 1
ω_2 = 1
Nbasis = 3
σx = sp.csr_matrix([[0.0,1.0],[1.0,0.0]])
σz = sp.csr_matrix([[1.0,0.0],[0.0,-1.0]])
σy = sp.csr_matrix([[0.0,-1.j],[1.j,0.0]])
σ_plus = sp.csr_matrix([[0.0, 1.0],[0.0, 0.0]])
σ_minus = σ_plus.T
σ_plus_minus = σ_plus@σ_minus

a = boson_anihilation(Nbasis)
adaga = boson_number(Nbasis)
adagaT = sp.kron(sp.eye(4),adaga)

In [ ]:
H0 = ω_1/2 * sp.kron(σz, sp.eye(2)) +  ω_2/2 * sp.kron(sp.eye(2), σz)
H0 = sp.kron(H0, sp.eye(Nbasis)) + ω_0 * adagaT 


In [ ]:
Hint = λ * ( sp.kron( sp.kron(σ_plus, sp.eye(2)), a ) + sp.kron( sp.kron(sp.eye(2), σ_plus), a ) )
Hint = Hint + Hint.T

In [ ]:
H = H0 + Hint

In [ ]:
Heff = H -1j*γ1*sp.kron(sp.kron(σ_plus_minus , sp.eye(2) ), sp.eye(Nbasis)   ) -1j*γ2*sp.kron(sp.kron( sp.eye(2), σ_plus_minus ), sp.eye(Nbasis)   ) 

In [ ]:
H = sp.csc_matrix(H)

In [ ]:
psin = [1,0,0]
ψ_init = np.kron(np.kron([1,0],[0,1]),psin)

Cn1 = sp.kron(sp.kron(σ_plus_minus, sp.eye(2)) , sp.eye(Nbasis))
Cn2 = sp.kron( sp.kron(sp.eye(2),σ_plus_minus) , sp.eye(Nbasis))
              
σ_minus1 = sp.kron( sp.kron(σ_minus, sp.eye(2)), sp.eye(Nbasis))
σ_minus2 = sp.kron( sp.kron(sp.eye(2),σ_minus), sp.eye(Nbasis))
              
σz1 = sp.kron(sp.kron(σz, sp.eye(2)) , sp.eye(Nbasis))
σz2 = sp.kron(sp.kron(sp.eye(2), σz)  ,sp.eye(Nbasis))

In [ ]:
def quantum_trajectories(Heff, ψ_init, Duration = 10, timesteps =100):

    time = np.linspace(0, Duration, timesteps)
    dt = time[1]-time[0]
    
    ψ = ψ_init
    
    boson_number = []
    Occupation_spin1 = []
    Occupation_spin2 = []
    
    Ut = scipy.linalg.expm(-1j * Heff * dt) 
    
    for t in time: 

        boson_number.append(  ψ.T.conjugate()@ (adagaT @ ψ) )

        Occupation_spin1.append( ψ.T.conjugate()  @ (σz1 @ ψ) )
        Occupation_spin2.append( ψ.T.conjugate() @  (σz2 @ ψ) )


        P1_jump =  dt * γ1 * ( ψ.T.conjugate() @ (Cn1 @ ψ) ) 
        P2_jump =  dt * γ2 * ψ.T.conjugate() @ (Cn2 @ ψ)

        if np.random.rand() <= P1_jump:
            ψ = σ_minus1 @ ψ
            ψ = ψ/np.sum(np.abs(ψ)**2)**0.5

        elif np.random.rand() <= P2_jump:
            ψ = σ_minus2 @ ψ
            ψ = ψ/np.sum(np.abs(ψ)**2)**0.5

        else:

            ψ = Ut @ ψ


            if (np.sum(np.abs(ψ)**2))**0.5 >= 1e-4:
                
                ψ = ψ/(np.sum(np.abs(ψ)**2))**0.5
    
    return np.real(boson_number), np.real(Occupation_spin1), np.real(Occupation_spin2), time

In [ ]:
def ensembles(Heff, ψ_init, Nr = 20, Duration = 10, timesteps =100):
    boson_number_mean = []
    Occupation_spin1_mean = []
    Occupation_spin2_mean = []
    for i in range(Nr):
        boson_number, Occupation_spin1, Occupation_spin2,_ =  quantum_trajectories(Heff = Heff, ψ_init = ψ_init, Duration = 10, timesteps =100)
        boson_number_mean.append(boson_number)
        Occupation_spin1_mean.append(Occupation_spin1)
        Occupation_spin2_mean.append(Occupation_spin2)


    boson_number_mean = np.sum(boson_number_mean, axis=0) / Nr
    Occupation_spin1_mean = np.sum(Occupation_spin1_mean, axis=0) / Nr
    Occupation_spin2_mean = np.sum(Occupation_spin2_mean, axis=0) / Nr

    return boson_number_mean, Occupation_spin1_mean, Occupation_spin2_mean

In [ ]:
ensembles1 = ensembles(Heff, ψ_init, Nr = 1)
ensembles10 = ensembles(Heff, ψ_init, Nr = 10)
ensembles1000 = ensembles(Heff, ψ_init, Nr = 1000)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows = 1, ncols = 3, figsize=(25,4))
time = np.linspace(0, 10, 100)
ax3.plot(time,ensembles1[0],':', label = '1 trajec')
ax3.plot(time,ensembles10[0],'-.', label = '5 trajec')
ax3.plot(time,ensembles1000[0], label = '1000 trajec')
ax3.set_ylabel('$\\langle a^{\dagger} a \\rangle$')
ax3.set_xlabel('time')

ax1.plot(time,ensembles1[1],':', label = '1 trajec')
ax1.plot(time,ensembles10[1],'-.', label = '5 trajec')
ax1.plot(time,ensembles1000[1], label = '1000 trajec')
ax1.set_ylabel('$\\langle \\sigma^z_1 \\rangle$')
ax1.set_xlabel('time')

ax2.plot(time,ensembles1[2],':', label = '1 trajec')
ax2.plot(time,ensembles10[2],'-.', label = '5 trajec')
ax2.plot(time,ensembles1000[2], label = '1000 trajec')
ax2.set_ylabel('$\\langle \\sigma^z_2 \\rangle$')
ax2.set_xlabel('time')

ax3.legend()

## Implementation in our physical setup

For that, we have to define a function which we will call "quantum_trajectory_time_dependent_H" that implements the evolution. This function substitutes the Trotter solver dynamics that had been used in the coherent case.

In [ ]:
def quantum_trajectory_time_dependent_H(control, ψ, duration, timesteps, γ1 = 1/12e3):

    time = np.linspace(-duration/2, duration/2, timesteps)
    
    ψt = [ψ]
    
    σ1_minus = erase(0, control.Basis)  # We construct the σ⁻ operator with our logic of conctenating bases
    σ1_plus = σ1_minus.T
    σ1_plus_minus = σ1_plus @ σ1_minus  # We define σ⁺σ⁻ which enters the P_jump and the H_eff
    
    last_t = time[0]
        
    for t in time[1:]: 
        
        dt = t - last_t
        
        P1_jump =  dt * γ1 * ( ψ.T.conjugate() @ (σ1_plus_minus @  ψ ) )  # Calculate the jump probability. Expected value of σ⁺σ⁻ times decay constant
        
        random = np.random.rand()
        
        if random <= P1_jump:  # If some random number is smaller than P1_jump, then proceed with the jump.
            
            print('QUANTUM JUMP at time =', t+duration/2)
            
            ψ = σ1_minus @ ψ  
            ψ = ψ/np.sum(np.abs(ψ)**2)**0.5  
            ψt.append(ψ)
 
        else:  # If no jump occurs, evolve the state under Heff.

            ψ = sp.linalg.expm_multiply((-1j*dt) * (control.Hamiltonian(t) - 1j * γ1/2 * σ1_plus_minus) , ψ)
            
            ψ = ψ/(np.sum(np.abs(ψ)**2))**0.5
            ψt.append(ψ)
            
        last_t = t
        
    return np.array(ψt).T #, np.real(boson_number), np.real(Occupation_spin1), np.real(Occupation_spin2), time

### Qubit T1

A check of weather the routines we are implementing to construct the hamiltonian and evolving it work in the many body simulation is to place an excitation in the qubit, isolated from everything else and let it evolve there. If everyting is well calibrated this should mimic an exponential decay $$e^{-\tau/T_1}, $$ with $T_1$ the decoherence time of the qubit. In our case $$T_1 = 12 \mu s. $$

In [6]:
''' Include vacuum  '''
Probe =  Exp_2qubits_2cavities_2purcells(δ1=2*π*8.40701933547913, δ2=2*π*8.40701933547913, g1=lambda t:0, g2=lambda t:0,
                                            ω1=2*π*8.40701933547913, ω2=2*π*8.407019335479,
                                            κ1 = 2*π*130e-3, κ2 = 2*π*130e-3, Nexcitations = 1, Up_to_Nexcitations = 1)

In [ ]:
ψ_init = np.zeros(len(Probe.Basis))
ψ_init[1] = 1
ψt = [quantum_trajectory_time_dependent_H(Probe, ψ_init, duration = 2000, timesteps =1000, γ1 = 1/12e3*10) for i in range(30)]

In [ ]:
P = np.abs(ψt)**2
time=np.linspace(0, 2000, 1000)
T1 = 12e3 
fig, ax = plt.subplots()
plt.plot(time, np.sum(P, axis = 0)[1,:]/(30), label = 'Approxiamtion with 30 Traject')
plt.plot(time, np.exp(-time/T1*10))
plt.ylabel('$|q_1(t)|^2$')
plt.xlabel('t (ns)')
plt.legend()
plt.show()

In [ ]:
# from joblib import delayed, Parallel, parallel_backend

# with parallel_backend('loky', n_jobs=36):
#     ψ_init = np.zeros(len(Probe.Basis))
#     ψ_init[1] = 1
#     ψt = Parallel()(delayed(quantum_trajectory_time_dependent_H)(Probe, ψ_init, duration = 200, timesteps =1000 ,
#                                                                 g1=lambda t:0,g2=lambda t:0,
#                                                                  γ1 = 1/12e3*10) for i in range(36) )


### Reabsorption protocol with quantum trajectories

Our idea is to replicate the results obtained by integration over the time the qubit was excited. This way we define the GOAL NUMBER, 
$$ \text{GOAL NUMBER} = |q_1(T)|^2 e^{-\tau/T_1} = 0.9822, $$
for a value of the decoherence time of $T_1 = 12 \mu s. $

The goal number was obtained by coherent evolution multiplied by the exponential decay, the objective is to replicate that number by averaging over different realizations by means of the Q-trajectories method.

In [8]:
def Bounce_and_reabsorb_trajectory(lamb_shift = (0.0229) * (2*π)*19.231e-3, δ1=2*π*8.40701933547913, δ2=2*π*8.40701933547913,
                   ω1=2*π*8.40701933547913, ω2=2*π*8.40701933547913,  g_p=2*π*25e-3,
                    κ1 = 2*π*130e-3, κ2 = 2*π*130e-3, η = 1, Tfp = 5, N_trips = 2, Nt=200,
                        length=30, modes=2101, Nexcitations = 1, extradelay = -3.3, quiet = False):
    
    kappa = κ1
    
    WG =  Waveguide( frequency= ω1+lamb_shift, modes = modes, length = length )

    κ_eff = 4 * g_p**2 / kappa 
    
    N_bounces = N_trips-1
    
    tdelay = N_trips * WG.tprop  +  N_bounces * 4 * (η/κ_eff)  + extradelay

    duration = tdelay + Tfp*4*(η/κ_eff)
    
    time = np.linspace(-duration/2, duration/2, Nt)

    def gtsym_sech(t):
        return gt_sech(t+tdelay/2.0, κ_eff) * (t <= 0) + gt_sech(tdelay/2.0 - t, κ_eff) * (t > 0)

    control = Exp_2qubits_2cavities_2purcells(δ1 = δ1, δ2 = δ2, ω1 = ω1, ω2 = ω2, g1=gtsym_sech, g2=0,  
                                                 gp1=g_p, gp2=g_p, κ1=κ1, κ2=κ2, 
                                             δLamb = lamb_shift, waveguide=WG, Nexcitations = Nexcitations, Up_to_Nexcitations = Nexcitations)

    ψ = np.zeros(len(control.Basis))
    ψ[1] = 1

    vt2 = quantum_trajectory_time_dependent_H(control, ψ, duration = duration, timesteps = Nt)
    
                
    ψ = vt2[:,-1]
    P = np.abs(vt2)**2

    q1end =np.abs(ψ[1])**2
    k = np.trapz(np.abs(vt2[0,:]**2), x = time)
    
    if not quiet:
        print('tdelay = ',tdelay,'(ns), of which,', extradelay,'is extra')
        print('Duration = ', duration, '(ns)')
        print(f'Initial value of the control g_1(-tf)= (2π)*{gtsym_sech(time[0])/2/π*1e3}MHz')
        # print(f'Time spent in qubit = {k} (ns)\n')

        Pwaveguide = np.sum(P[6+1:,:],0)
        print(f'|q_1(tf)|^2 = {P[0+1,-1]}\n'
              f'|c_1(tf)|^2 = {P[2+1,-1]}\n'
              f'|Purcell_1(tf)|^2 = {P[4+1,-1]}\n'
              f'|wv(tf)|^2  = {Pwaveguide[-1]} (max = {max(Pwaveguide)})\n'
              f'|Purcell_2(tf)|^2 = {P[5+1,-1]}\n'  
              f'|c_2(tf)|^2 = {P[3+1,-1]}\n'
              f'|q_2(tf)|^2 = {P[1+1,-1]}')
              # f' 1-F        = {1-P[1,-1]}')

        Pwaveguide = np.sum(P[6+1:,:],0)
        fig, (ax) = plt.subplots( figsize=(10,3.5), sharex=True)
        ax.plot(time, P[0+1 ,:], label='qubit 1')
        ax.plot(time, P[2+1,:], '--', label='cavity 1')
        ax.plot(time, P[4+1,:], '--', label='Purcell 1')

        ax.plot(time, Pwaveguide, '-.', label='waveguide')

        ax.plot(time, P[5+1,:], '--', label='Purcell 2')
        ax.plot(time, P[3+1,:], '--', label='cavity 2')

        ax.legend()
        fig.tight_layout()
        plt.show()
    
    g0 = gtsym_sech(time[0])/2/π*1e3

    return q1end#, time, g0, k, duration, P

In [9]:
def gt_sech(t, κ):
    
    return κ / 2 / np.cosh(κ*t/2)

In [ ]:
with parallel_backend('loky', n_jobs=36):
    Bounce_and_back = Parallel()(delayed(Bounce_and_reabsorb_trajectory)(extradelay = -3.3, Tfp = 5.2, Nm=1101,
                                                                         Nt=1000, quiet = True) for i in range(36*20) )


In [ ]:
Duration =  520.3496356160991 (ns)
#Initial value of the control g_1(-tf)= (2π)*0.11064634090648054MHz
τ = 151.9227183160247

In [ ]:
T1 = 12e3 
Protocol_time_eta_1 = 520.12265222192
τ = 151.9227183160247
Q2_end = 0.99419
GOAL_NUMBER = 0.99419 * np.exp(-τ/T1)
fix, ax1 = plt.subplots()
ax1.plot(Protocol_time_eta_1, Q2_end,'o',  label='Ideal')
ax1.plot(Protocol_time_eta_1, GOAL_NUMBER,'o', label='Decoherence. $T_1 = 12 \mu s.$')
ax1.plot(Protocol_time_eta_1, np.sum(Bounce_and_back, axis = 0)/(36*20),'o',label='720 Quantum trajectories')
ax1.set_ylabel('$|q_1(t)|^2$')
ax1.set_xlabel('t (ns)')

ax1.legend()
plt.show()

In [ ]:
print('Coherent evolution = ', np.round(Q2_end,4))
print('Decoherence with T_1 = ', np.round(GOAL_NUMBER,4))
print('Average over',36*20,'quantum trajectories = ', np.round(np.sum(Bounce_and_back, axis = 0)/(36*20),4))
#Coherent evolution =  0.9942
#Decoherence with T_1 =  0.9817
#Average over 720 quantum trajectories =  0.9818
